In [6]:
import requests
import json
import hashlib
from brownie import convert

from ckb_toolkit.core.signer import sign_tx
from ckb_toolkit.core.rpc import RPCClient
from coincurve import PublicKey, PrivateKey
from ckb_toolkit.core.hash import ckb_blake160
from ckb_toolkit.core.hex_coder import hex_to_bytes, hex_from_bytes
from ckb_toolkit.address.address import generateShortAddress, generateFullAddress

rpc_url = 'https://testnet.ckbapp.dev'
rpc = RPCClient(rpc_url)
priv_key = ""
TESTNET_SECP256K1_BLAKE160_CODE_HASH = "0x9bd7e06f3ecf4be0f2fcd2188b23f1b9fcc88e5d4b65a8637b17723bbda3cce8"

user_alice_args = "0x5f58632a123e7e731db0b59ce50fe16579c7d941"
user_bob_args = "0x9a375c86a83af9ade872c4ff85033b761b33caf2"

def packNumber(num):
    return bytes.fromhex(convert.to_uint(num,"uint8").to_bytes(4,'little').hex())

def script_hash(code_hash = "c5e5dcf215925f7ef4dfaf5f4b4f105bc321c02776d6e7d52a1db3fcd9d011a4", hash_type = "01", args = "7e9937fdf99f952ab8ea150a181fc7266e8976173d5cf201a13895e4ea2e457b"):
    code_hash_b = bytes.fromhex(code_hash)
    hash_type_b = bytes.fromhex(hash_type)
    args_b = b"\x20\x00\x00\x00" + bytes.fromhex(args)
    len_of_all = 16 + len(code_hash_b) + len(hash_type_b) + len(args_b)

    bytes_all = packNumber(len_of_all)\
        + packNumber(16)\
            + packNumber(16 + len(code_hash_b))\
                + packNumber(16 + len(code_hash_b) + len(hash_type_b))\
                    + code_hash_b\
                        + hash_type_b\
                            + args_b

    hasher = hashlib.blake2b(digest_size=32, person=b'ckb-default-hash')
    hasher.update(bytes_all)
    return '0x' + hasher.digest().hex()

# 全链搜索使用了script的cell
def get_cells_by_script(code_hash = "0x3419a1c09eb2567f6552ee7a8ecffd64155cffe0f1796e6e61ec088d740c1356", hash_type = "type", args = "0x", script_type = "lock"):
    json_data = {
        'jsonrpc': '2.0',
        'method': 'get_cells',
        'params': [
            {
                "script": {
                    "code_hash": code_hash, #"0x3419a1c09eb2567f6552ee7a8ecffd64155cffe0f1796e6e61ec088d740c1356",
                    "hash_type": hash_type, #"type",
                    "args": args, #"0x"
                },
                "script_type": script_type, #"lock"
            },
            "asc",
            "0x64"
        ],
        'id': 2,
    }

    response = requests.post(rpc_url, json=json_data)
    result = json.loads(response.content)
    return result

# 从文件导入cells
def load_cells_from_file(file_path):
    items = []
    with open(file_path) as f:
        js = json.loads(f.read())
        for item in js['result']['objects']:
          items.append(item)
    return items

# 获得某个scripts 封印的所有cell 的capacity总和，一般用于查一下拥有多少ckb
def get_cells_capacity():
    json_data = {
        'jsonrpc': '2.0',
        'method': 'get_cells_capacity',
        "params": [
            {
                "script": {
                    "code_hash": "0x9bd7e06f3ecf4be0f2fcd2188b23f1b9fcc88e5d4b65a8637b17723bbda3cce8",
                    "hash_type": "type",
                    "args": "0x5f58632a123e7e731db0b59ce50fe16579c7d941"
                },
                "script_type": "lock"
            }
        ],
        'id': 1,
    }

    response = requests.post(rpc_url, json=json_data)
    return json.loads(response.content)

# get_transaction 
def get_transaction(txhash):
    json_data = {
        'jsonrpc': '2.0',
        'method': 'get_transaction',
        'params': [
            txhash,
        ],
        'id': 1,
    }

    response = requests.post(rpc_url, json=json_data)
    return json.loads(response.content)

# get cell capacity # 主要用于获取input的capacity有多少
def get_capacity_of_cell(cell_info):
    index = int(cell_info['index'], 16)
    tx_hash = cell_info['tx_hash']

    tx = get_transaction(tx_hash)
    capacity = 0
    if tx.get("result") != None:
        output_cell = tx["result"]["transaction"]['outputs'][index]
        capacity = output_cell['capacity']
    return capacity

# 发送transaction
''' cell_deps #合约代码cell
[
    {
        'dep_type': 'dep_group',
        'out_point': {
            'index': '0x0',
            'tx_hash': '0xf8de3bb47d055cdf460d93a2a6e1b05f7432f9777c8c474abf4eec1d4aee5d37',
        },
    },
]
'''

''' header_deps #交易中可能涉及的区块
[]
'''

''' inputs #成本cell 要解除封印的cell
[
    {
        'previous_output': {
            'index': '0x0',
            'tx_hash': '0xb276d7bb2d4b94361bb6bd4a4c08c50aff649ed15d4aa2a0f8db7df86268dc39',
        },
        'since': '0x0',
    },
]
'''

''' outputs #输出cell 
[
    {
        'capacity': '0x48c27395000',
        'lock': {
            'args': '0xff5094c2c5f476fc38510018609a3fd921dd28ad',
            'code_hash': '0x9bd7e06f3ecf4be0f2fcd2188b23f1b9fcc88e5d4b65a8637b17723bbda3cce8',
            'hash_type': 'type',
        },
        'type': None,
    },
]
'''

def send_transaction(inputs, cell_deps, outputs, header_deps = [], outputs_data = ['0x'], witnesses = ['0x']):
    raw_tx = {
        'cell_deps': cell_deps,
        'header_deps': header_deps,
        'inputs': inputs,
        'outputs': outputs,
        'outputs_data': outputs_data,
        'version': '0x0',
        'witnesses': witnesses,
    }

    tx = sign_tx(raw_tx, priv_key)
    print(json.dumps(tx, indent=4))

    json_data = {
        'jsonrpc': '2.0',
        'method': 'send_transaction',
        'params': [tx],
        'id': 1,
    }

    response = requests.post(rpc_url, json=json_data)
    result = json.loads(response.content)
    return result
    
# 使用部署好的 script_info + args，封印 inputs_cell
# script_info 是官方explorer中使用的script结构体 https://pudge.explorer.nervos.org/scripts
''' script_info demo
{
  "code_hash": "0x9bd7e06f3ecf4be0f2fcd2188b23f1b9fcc88e5d4b65a8637b17723bbda3cce8",
  "hash_type": "type",
  "out_point": {
    "tx_hash": "0xf8de3bb47d055cdf460d93a2a6e1b05f7432f9777c8c474abf4eec1d4aee5d37",
    "index": "0x0"
  },
  "dep_type": "dep_group"
}
'''

# 解封ckb 用script_info 进行lock封印
def seal_normal_cell_and_lock_with_script_info(inputs, script_info, args):
    #script_info
    code_hash = script_info['code_hash']
    hash_type = script_info['hash_type']

    if code_hash == TESTNET_SECP256K1_BLAKE160_CODE_HASH:
        cell_deps = [{
            "out_point": script_info['out_point'],
            "dep_type": script_info['dep_type']
        }]
    else:
        cell_deps = [{
            "out_point": script_info['out_point'],
            "dep_type": script_info['dep_type']
        },{
            "out_point": {
                "tx_hash": "0xf8de3bb47d055cdf460d93a2a6e1b05f7432f9777c8c474abf4eec1d4aee5d37",
                "index": "0x0"
            },
            "dep_type": "dep_group"
        }]

    inputs_capacity = []
    input_output_points = []
    for input_cell in inputs['result']['objects']:
        capacity = int(input_cell['output']['capacity'], 16)
        inputs_capacity.append(capacity)

        output_point = {
            "previous_output": input_cell['out_point'],
            "since": "0x0",
        }
        input_output_points.append(output_point)
    
    sum_capacity = sum(inputs_capacity)
    outputs = [
        {
            'capacity': hex(sum_capacity - 1000000),
            'lock': {
                'args': args,
                'code_hash': code_hash,
                'hash_type': hash_type,
            },
            'type': None,
        },
    ]

    return send_transaction(input_output_points, cell_deps, outputs)

# 解封ckb 用script_info 进行type封印
def seal_normal_cell_and_type_with_script_info(inputs, script_info, args):
    #script_info
    code_hash = script_info['code_hash']
    hash_type = script_info['hash_type']

    if code_hash == TESTNET_SECP256K1_BLAKE160_CODE_HASH:
        cell_deps = [{
            "out_point": script_info['out_point'],
            "dep_type": script_info['dep_type']
        }]
    else:
        cell_deps = [{
            "out_point": script_info['out_point'],
            "dep_type": script_info['dep_type']
        },{
            "out_point": {
                "tx_hash": "0xf8de3bb47d055cdf460d93a2a6e1b05f7432f9777c8c474abf4eec1d4aee5d37",
                "index": "0x0"
            },
            "dep_type": "dep_group"
        }]

    inputs_capacity = []
    input_output_points = []
    for input_cell in inputs['result']['objects']:
        capacity = int(input_cell['output']['capacity'], 16)
        inputs_capacity.append(capacity)

        output_point = {
            "previous_output": input_cell['out_point'],
            "since": "0x0",
        }
        input_output_points.append(output_point)
    
    sum_capacity = sum(inputs_capacity)
    outputs = [
        {
            'capacity': hex(sum_capacity - 1000000),
            'type': {
                'args': args,
                'code_hash': code_hash,
                'hash_type': hash_type,
            },
            'lock': None,
        },
    ]

    return send_transaction(input_output_points, cell_deps, outputs)

# 全部ckb转账
def transfer(from_args="0x9a375c86a83af9ade872c4ff85033b761b33caf2", to_args="0x5f58632a123e7e731db0b59ce50fe16579c7d941"):
  script_info = {
    "code_hash": "0x9bd7e06f3ecf4be0f2fcd2188b23f1b9fcc88e5d4b65a8637b17723bbda3cce8",
    "hash_type": "type",
    "out_point": {
      "tx_hash": "0xf8de3bb47d055cdf460d93a2a6e1b05f7432f9777c8c474abf4eec1d4aee5d37",
      "index": "0x0"
    },
    "dep_type": "dep_group"
  }

  # ckb inputs
  code_hash = "0x9bd7e06f3ecf4be0f2fcd2188b23f1b9fcc88e5d4b65a8637b17723bbda3cce8"
  hash_type = "type"
  args = from_args
  script_type = "lock"
  input_cells = get_cells_by_script(code_hash=code_hash, hash_type=hash_type, args=args, script_type=script_type)
  for i in range(0, len(input_cells['result']['objects'])):
    if input_cells['result']['objects'][i]['output']['lock']['args'] != args:
        print(input_cells['result']['objects'][i]['output']['lock']['args'])
        input_cells['result']['objects'].pop(i)
  return seal_normal_cell_and_lock_with_script_info(input_cells, script_info, to_args)

# mint sudt
# mint_sudt(10000)
def mint_sudt(amount, to_secpk256c1_args="0x5f58632a123e7e731db0b59ce50fe16579c7d941"):
  # sudt script info 
  script_info = {
    "code_hash": "0xc5e5dcf215925f7ef4dfaf5f4b4f105bc321c02776d6e7d52a1db3fcd9d011a4",
    "hash_type": "type",
    "out_point": {
      "tx_hash": "0xe12877ebd2c3c364dc46c5c992bcfaf4fee33fa13eebdf82c591fc9825aab769",
      "index": "0x0"
    },
    "dep_type": "code"
  }

  # ckb inputs
  code_hash = "0x9bd7e06f3ecf4be0f2fcd2188b23f1b9fcc88e5d4b65a8637b17723bbda3cce8"
  hash_type = "type"
  args = to_secpk256c1_args
  script_type = "lock"
  input_cells = get_cells_by_script(code_hash=code_hash, hash_type=hash_type, args=args, script_type=script_type)

  # cells 除杂
  for i in range(0, len(input_cells['result']['objects'])):
    if input_cells['result']['objects'][i]['output']['lock']['args'] != args:
      print(input_cells['result']['objects'][i]['output']['lock']['args'])
      input_cells['result']['objects'].pop(i)

  # ckb -> sudt
  #sudt_name_hex = sudt_name.encode("utf-8").hex()
  owner_lock_script_hash = "0x7e9937fdf99f952ab8ea150a181fc7266e8976173d5cf201a13895e4ea2e457b"
  sudt_arg = owner_lock_script_hash #+ sudt_name_hex
  #tx_result = seal_normal_cell_and_lock_with_script_info(input_cells, script_info, sudt_arg)

  cell_deps = [{
    "out_point": {
      "tx_hash": "0xe12877ebd2c3c364dc46c5c992bcfaf4fee33fa13eebdf82c591fc9825aab769",
      "index": "0x0"
    },
    "dep_type": "code"
  },{
    "out_point": {
      "tx_hash": "0xf8de3bb47d055cdf460d93a2a6e1b05f7432f9777c8c474abf4eec1d4aee5d37",
      "index": "0x0"
    },
    "dep_type": "dep_group"
  }]
  inputs_capacity = []
  input_output_points = []
  for input_cell in input_cells['result']['objects']:
    capacity = int(input_cell['output']['capacity'], 16)
    inputs_capacity.append(capacity)

    output_point = {
      "previous_output": input_cell['out_point'],
      "since": "0x0",
    }
    input_output_points.append(output_point)
  
  sum_capacity = sum(inputs_capacity)
  outputs = [
    {
      'capacity': hex(17400000000),
      'type': {
        'code_hash': script_info['code_hash'],
        'hash_type': script_info['hash_type'],
        'args': sudt_arg
      },
      'lock': {
        'code_hash': '0x9bd7e06f3ecf4be0f2fcd2188b23f1b9fcc88e5d4b65a8637b17723bbda3cce8',
        'hash_type': "type",
        'args': to_secpk256c1_args
      },
    },
    {
      'capacity': hex(sum_capacity - 100000 - 17400000000),
      'type': None,
      'lock': {
        'code_hash': '0x9bd7e06f3ecf4be0f2fcd2188b23f1b9fcc88e5d4b65a8637b17723bbda3cce8',
        'hash_type': "type",
        'args': to_secpk256c1_args
      },
    }
  ]

  amount_hex = '0x' + convert.to_uint("10000","uint128").to_bytes(16,'little').hex()

  tx_result = send_transaction(input_output_points, cell_deps, outputs, outputs_data=[amount_hex,"0x"])
  return tx_result

# sudt + CKB -> sudt + sudt + CKB 转账
# transfer_sudt_with_ckb_fee("0x5f58632a123e7e731db0b59ce50fe16579c7d941", "0x9a375c86a83af9ade872c4ff85033b761b33caf2", "0x7e9937fdf99f952ab8ea150a181fc7266e8976173d5cf201a13895e4ea2e457b", 10)
def transfer_sudt_with_ckb_fee(from_secpk256c1_args, to_secpk256c1_args, sudt_typescript_args, transfer_amount):
    # from 有用的所有cells
    code_hash = "0x9bd7e06f3ecf4be0f2fcd2188b23f1b9fcc88e5d4b65a8637b17723bbda3cce8"
    hash_type = "type"
    args = from_secpk256c1_args
    script_type = "lock"
    input_cells = get_cells_by_script(code_hash=code_hash, hash_type=hash_type, args=args, script_type=script_type)

    objects = []
    ckb_cell_input = None
    for i, cell in enumerate(input_cells['result']['objects']):
        if cell['output']['type'] == None:
            if cell['output']['lock']['args'] == from_secpk256c1_args and \
                cell['output']['lock']['code_hash'] == '0x9bd7e06f3ecf4be0f2fcd2188b23f1b9fcc88e5d4b65a8637b17723bbda3cce8':
                  ckb_cell_input = cell
            continue
        if cell['output']['type']['args'] == sudt_typescript_args\
            and cell['output']['type']['code_hash'] == "0xc5e5dcf215925f7ef4dfaf5f4b4f105bc321c02776d6e7d52a1db3fcd9d011a4" :
            objects.append(cell)
    
    input_cells['result']['objects'] = objects

    cell_deps = [{
      "out_point": {
        "tx_hash": "0xe12877ebd2c3c364dc46c5c992bcfaf4fee33fa13eebdf82c591fc9825aab769",
        "index": "0x0"
      },
      "dep_type": "code"
    },{
      "out_point": {
        "tx_hash": "0xf8de3bb47d055cdf460d93a2a6e1b05f7432f9777c8c474abf4eec1d4aee5d37",
        "index": "0x0"
      },
      "dep_type": "dep_group"
    }]
    inputs_capacity = []
    input_output_points = []
    for input_cell in input_cells['result']['objects']:
      capacity = int(input_cell['output']['capacity'], 16)
      inputs_capacity.append(capacity)

      output_point = {
        "previous_output": input_cell['out_point'],
        "since": "0x0",
      }
      input_output_points.append(output_point)
    
    input_output_points.append({
      "previous_output":ckb_cell_input['out_point'],
      "since": "0x0"})
    print(input_output_points)
    
    sum_capacity = sum(inputs_capacity) + int(ckb_cell_input['output']['capacity'],16)
    outputs = [
      {
        'capacity': hex(17400000000),
        'type': {
          'code_hash': "0xc5e5dcf215925f7ef4dfaf5f4b4f105bc321c02776d6e7d52a1db3fcd9d011a4",
          'hash_type': "type",
          'args': sudt_typescript_args
        },
        'lock': {
          'code_hash': '0x9bd7e06f3ecf4be0f2fcd2188b23f1b9fcc88e5d4b65a8637b17723bbda3cce8',
          'hash_type': "type",
          'args': from_secpk256c1_args
        },
      },
      {
        'capacity': hex(17400000000),
        'type': {
          'code_hash': "0xc5e5dcf215925f7ef4dfaf5f4b4f105bc321c02776d6e7d52a1db3fcd9d011a4",
          'hash_type': "type",
          'args': sudt_typescript_args
        },
        'lock': {
          'code_hash': '0x9bd7e06f3ecf4be0f2fcd2188b23f1b9fcc88e5d4b65a8637b17723bbda3cce8',
          'hash_type': "type",
          'args': to_secpk256c1_args
        },
      },
      {
        'capacity': hex(sum_capacity - 100000 - 17400000000 * 2),
        'type': None,
        'lock': {
          'code_hash': '0x9bd7e06f3ecf4be0f2fcd2188b23f1b9fcc88e5d4b65a8637b17723bbda3cce8',
          'hash_type': "type",
          'args': from_secpk256c1_args
        },
      }
    ]

    amounts = []
    for cell in input_cells['result']['objects']:
          amount_string: str = cell['output_data']
          amount = int.from_bytes(bytes.fromhex(amount_string[2:]), byteorder='little') 
          amounts.append(amount)
    sudt_amount_sum = sum(amounts)

    amount_to = '0x' + convert.to_uint(transfer_amount,"uint128").to_bytes(16,'little').hex()
    amount_from = '0x' + convert.to_uint(sudt_amount_sum - transfer_amount,"uint128").to_bytes(16,'little').hex()

    tx_result = send_transaction(input_output_points, cell_deps, outputs, outputs_data=[amount_from, amount_to,"0x"])
    return tx_result
    
# sudt -> sudt 转账
# transfer_sudt("0x9a375c86a83af9ade872c4ff85033b761b33caf2", "0x5f58632a123e7e731db0b59ce50fe16579c7d941", "0x7e9937fdf99f952ab8ea150a181fc7266e8976173d5cf201a13895e4ea2e457b")
def transfer_sudt(from_secpk256c1_args, to_secpk256c1_args, sudt_typescript_args):
    # from 有用的所有cells
    code_hash = "0x9bd7e06f3ecf4be0f2fcd2188b23f1b9fcc88e5d4b65a8637b17723bbda3cce8"
    hash_type = "type"
    args = from_secpk256c1_args
    script_type = "lock"
    input_cells = get_cells_by_script(code_hash=code_hash, hash_type=hash_type, args=args, script_type=script_type)

    objects = []
    for i, cell in enumerate(input_cells['result']['objects']):
        if cell['output']['type'] == None:
            continue
        if cell['output']['type']['args'] == sudt_typescript_args\
            and cell['output']['type']['code_hash'] == "0xc5e5dcf215925f7ef4dfaf5f4b4f105bc321c02776d6e7d52a1db3fcd9d011a4" :
            objects.append(cell)
    
    input_cells['result']['objects'] = objects

    cell_deps = [{
      "out_point": {
        "tx_hash": "0xe12877ebd2c3c364dc46c5c992bcfaf4fee33fa13eebdf82c591fc9825aab769",
        "index": "0x0"
      },
      "dep_type": "code"
    },{
      "out_point": {
        "tx_hash": "0xf8de3bb47d055cdf460d93a2a6e1b05f7432f9777c8c474abf4eec1d4aee5d37",
        "index": "0x0"
      },
      "dep_type": "dep_group"
    }]
    inputs_capacity = []
    input_output_points = []
    for input_cell in input_cells['result']['objects']:
      capacity = int(input_cell['output']['capacity'], 16)
      inputs_capacity.append(capacity)

      output_point = {
        "previous_output": input_cell['out_point'],
        "since": "0x0",
      }
      input_output_points.append(output_point)
    
    sum_capacity = sum(inputs_capacity)
    outputs = [
      {
        'capacity': hex(sum_capacity - 100000),
        'type': {
          'code_hash': "0xc5e5dcf215925f7ef4dfaf5f4b4f105bc321c02776d6e7d52a1db3fcd9d011a4",
          'hash_type': "type",
          'args': sudt_typescript_args
        },
        'lock': {
          'code_hash': '0x9bd7e06f3ecf4be0f2fcd2188b23f1b9fcc88e5d4b65a8637b17723bbda3cce8',
          'hash_type': "type",
          'args': to_secpk256c1_args
        },
      }
    ]

    amounts = []
    for cell in input_cells['result']['objects']:
          amount_string: str = cell['output_data']
          amount = int.from_bytes(bytes.fromhex(amount_string[2:]), byteorder='little') 
          amounts.append(amount)
    sudt_amount_sum = sum(amounts)

    amount_to = '0x' + convert.to_uint(sudt_amount_sum,"uint128").to_bytes(16,'little').hex()

    tx_result = send_transaction(input_output_points, cell_deps, outputs, outputs_data=[amount_to])
    return tx_result

# sudt10 -> sudt0 burn
# burn_sudt(10, "0x7e9937fdf99f952ab8ea150a181fc7266e8976173d5cf201a13895e4ea2e457b")
def burn_sudt(amount, sudt_typescript_args):
    public_key = PrivateKey.from_hex(priv_key).public_key.format()
    args = ckb_blake160(public_key)
    # from 有用的所有cells
    code_hash = "0x9bd7e06f3ecf4be0f2fcd2188b23f1b9fcc88e5d4b65a8637b17723bbda3cce8"
    hash_type = "type"
    script_type = "lock"
    input_cells = get_cells_by_script(code_hash=code_hash, hash_type=hash_type, args=args, script_type=script_type)

    objects = []
    for i, cell in enumerate(input_cells['result']['objects']):
        if cell['output']['type'] == None:
            continue
        if cell['output']['type']['args'] == sudt_typescript_args\
            and cell['output']['type']['code_hash'] == "0xc5e5dcf215925f7ef4dfaf5f4b4f105bc321c02776d6e7d52a1db3fcd9d011a4" :
            objects.append(cell)
    
    input_cells['result']['objects'] = objects

    cell_deps = [{
      "out_point": {
        "tx_hash": "0xe12877ebd2c3c364dc46c5c992bcfaf4fee33fa13eebdf82c591fc9825aab769",
        "index": "0x0"
      },
      "dep_type": "code"
    },{
      "out_point": {
        "tx_hash": "0xf8de3bb47d055cdf460d93a2a6e1b05f7432f9777c8c474abf4eec1d4aee5d37",
        "index": "0x0"
      },
      "dep_type": "dep_group"
    }]
    inputs_capacity = []
    input_output_points = []
    for input_cell in input_cells['result']['objects']:
      capacity = int(input_cell['output']['capacity'], 16)
      inputs_capacity.append(capacity)

      output_point = {
        "previous_output": input_cell['out_point'],
        "since": "0x0",
      }
      input_output_points.append(output_point)
    
    sum_capacity = sum(inputs_capacity)
    outputs = [
      {
        'capacity': hex(sum_capacity - 100000),
        'type': {
          'code_hash': "0xc5e5dcf215925f7ef4dfaf5f4b4f105bc321c02776d6e7d52a1db3fcd9d011a4",
          'hash_type': "type",
          'args': sudt_typescript_args
        },
        'lock': {
          'code_hash': '0x9bd7e06f3ecf4be0f2fcd2188b23f1b9fcc88e5d4b65a8637b17723bbda3cce8',
          'hash_type': "type",
          'args': args
        },
      }
    ]

    amounts = []
    for cell in input_cells['result']['objects']:
          amount_string: str = cell['output_data']
          amount = int.from_bytes(bytes.fromhex(amount_string[2:]), byteorder='little') 
          amounts.append(amount)
    sudt_amount_sum = sum(amounts)

    amount_to = '0x' + convert.to_uint(sudt_amount_sum - amount,"uint128").to_bytes(16,'little').hex()

    tx_result = send_transaction(input_output_points, cell_deps, outputs, outputs_data=[amount_to])
    return tx_result

# 解封ACP Owner
def unseal_acp_into_ckb(inputs, output_args):
    #script_info
    code_hash = TESTNET_SECP256K1_BLAKE160_CODE_HASH
    hash_type = 'type'

    cell_deps = [{ # 这个是acp的dep cell
        "out_point": {
            "tx_hash": "0xec26b0f85ed839ece5f11c4c4e837ec359f5adc4420410f6453b1f6b60fb96a6",
            "index": "0x0"
        },
        "dep_type": "dep_group"
    },{ # secp256K1
        "out_point": {
            "tx_hash": "0xf8de3bb47d055cdf460d93a2a6e1b05f7432f9777c8c474abf4eec1d4aee5d37",
            "index": "0x0"
        },
        "dep_type": "dep_group"
    }]

    inputs_capacity = []
    input_output_points = []
    for input_cell in inputs['result']['objects']:
        capacity = int(input_cell['output']['capacity'], 16)
        inputs_capacity.append(capacity)

        output_point = {
            "previous_output": input_cell['out_point'],
            "since": "0x0",
        }
        input_output_points.append(output_point)
    
    sum_capacity = sum(inputs_capacity)
    outputs = [
        {
            'capacity': hex(sum_capacity - 1000000),
            'lock': {
                'args': output_args,
                'code_hash': code_hash,
                'hash_type': hash_type,
            },
            'type': None,
        },
    ]

    return send_transaction(input_output_points, cell_deps, outputs)

# to ACP 
# output_point = {
#     "tx_hash": "0x3f1ee5f734316c1d5f3bbb0810b45e271c9eb0b166446b3927860c8243d77b57",
#     "index": "0x0"
# }

# unseal_acp_into_ckb(output_point)
def unseal_acp_into_ckb(inputs):
    public_key = PrivateKey.from_hex(priv_key).public_key.format()
    output_args = ckb_blake160(public_key)
    #script_info
    code_hash = TESTNET_SECP256K1_BLAKE160_CODE_HASH
    hash_type = 'type'

    cell_deps = [{ # 这个是acp的dep cell
        "out_point": {
            "tx_hash": "0xec26b0f85ed839ece5f11c4c4e837ec359f5adc4420410f6453b1f6b60fb96a6",
            "index": "0x0"
        },
        "dep_type": "dep_group"
    },{ # secp256K1
        "out_point": {
            "tx_hash": "0xf8de3bb47d055cdf460d93a2a6e1b05f7432f9777c8c474abf4eec1d4aee5d37",
            "index": "0x0"
        },
        "dep_type": "dep_group"
    }]

    input_output_point = {
        "previous_output" : inputs,
        "since": "0x0"
    }
    
    input_output_points = []
    input_output_points.append(input_output_point)
    
    inputs_capacity = int(get_capacity_of_cell(inputs['previous_output']), 16)

    outputs = [
        {
            'capacity': hex(6200000000),
            'lock': {
                'args': output_args + "09",
                'code_hash': "0x3419a1c09eb2567f6552ee7a8ecffd64155cffe0f1796e6e61ec088d740c1356",
                'hash_type': "type",
            },
            'type': None # 这里搞个sudt可以用来接收SUDT，但是这里就先不了。
        },
        {
            'capacity': hex(inputs_capacity - 6200000000 - 1000000),
            'lock': {
                'args': output_args,
                'code_hash': code_hash,
                'hash_type': hash_type,
            },
            'type': None,
        },
    ]

    return send_transaction(input_output_points, cell_deps, outputs, outputs_data=['0x','0x'])

# stake n < 61 ckb into acp
# acp_input = {
#     "tx_hash": "0x557fa7bd6e96a0769d28888561e6186ac9082c622ab0c751109ba965d1d28741",
#     "index": "0x0"
# }

# ckb_input = {
#     "tx_hash": "0x557fa7bd6e96a0769d28888561e6186ac9082c622ab0c751109ba965d1d28741",
#     "index": "0x1"
# }

# stakeCKBintoACP(acp_input, "0x5f58632a123e7e731db0b59ce50fe16579c7d94109", ckb_input, 10)
def stakeCKBintoACP(acp_input, acp_cell_args, ckb_input,  min_amount):
    public_key = PrivateKey.from_hex(priv_key).public_key.format()
    output_args = ckb_blake160(public_key)
    #script_info
    code_hash = TESTNET_SECP256K1_BLAKE160_CODE_HASH
    hash_type = 'type'

    cell_deps = [{ # 这个是acp的dep cell
        "out_point": {
            "tx_hash": "0xec26b0f85ed839ece5f11c4c4e837ec359f5adc4420410f6453b1f6b60fb96a6",
            "index": "0x0"
        },
        "dep_type": "dep_group"
    },{ # secp256K1
        "out_point": {
            "tx_hash": "0xf8de3bb47d055cdf460d93a2a6e1b05f7432f9777c8c474abf4eec1d4aee5d37",
            "index": "0x0"
        },
        "dep_type": "dep_group"
    }]
    
    input_output_points = []
    input_output_points.append({
        "previous_output" : ckb_input,
        "since": "0x0"
    })
    input_output_points.append({
        "previous_output" : acp_input,
        "since": "0x0"
    })
    
    ckb_capacity_old = int(get_capacity_of_cell(ckb_input), 16)
    acp_capacity_old = int(get_capacity_of_cell(acp_input), 16)

    outputs = [
        {
            'capacity': hex(acp_capacity_old + int(min_amount*1e8)),
            'lock': {
                'args': acp_cell_args,
                'code_hash': "0x3419a1c09eb2567f6552ee7a8ecffd64155cffe0f1796e6e61ec088d740c1356",
                'hash_type': "type",
            },
            'type': None # 这里搞个sudt可以用来接收SUDT，但是这里就先不了。
        },
        {
            'capacity': hex(ckb_capacity_old - int(min_amount*1e8) - 1000000),
            'lock': {
                'args': output_args,
                'code_hash': code_hash,
                'hash_type': hash_type,
            },
            'type': None,
        },
    ]

    return send_transaction(input_output_points, cell_deps, outputs, outputs_data=['0x','0x'])